<a href="https://colab.research.google.com/github/Tsocode/Currency-Buddy/blob/master/Speech_Recogintion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install torch torchaudio transformers --quiet


In [17]:
import torch
import torchaudio
from google.colab import files
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

try:
    # Load the pre-trained Wav2Vec 2.0 model and processor from Hugging Face
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    model.eval()  # Set the model to evaluation mode
except Exception as e:
    print(f"Error loading model or processor: {e}")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You sho

In [18]:
def transcribe_with_deep_learning(audio_path):
    try:
        waveform, sample_rate = torchaudio.load(audio_path)

        # Resample the waveform to the sample rate the model was trained on (16kHz for Wav2Vec2)
        if sample_rate != 16000:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            waveform = resampler(waveform)

        # Process the waveform through the model
        input_values = processor(waveform, return_tensors="pt", sampling_rate=16000).input_values
        with torch.no_grad():
            logits = model(input_values).logits

        # Decode the model output
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)

        return transcription[0]
    except Exception as e:
        return f"Error during transcription: {e}"


In [19]:
# Use the Colab file upload mechanism.
uploaded_files = files.upload()

# List all uploaded files (optional)
for fn in uploaded_files.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded_files[fn])))


Saving TranscriptAudio.m4a to TranscriptAudio (2).m4a
User uploaded file "TranscriptAudio (2).m4a" with length 42906 bytes


In [20]:
# Loop through each uploaded file and transcribe
for filename in uploaded_files.keys():
    print(f"\nTranscribing file: {filename}")
    try:
        transcription = transcribe_with_deep_learning(filename)
        print("Transcription:", transcription)
    except Exception as e:
        print(f"Failed to transcribe {filename}: {e}")



Transcribing file: TranscriptAudio (2).m4a
Transcription: Error during transcription: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [1, 1, 1, 76459]
